In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from dateutil.relativedelta import relativedelta

In [2]:
ER = pd.read_excel('all_data/Bayes_stein_returns.xlsx')
garch_vol = pd.read_excel('all_data/garch_volatility_forecasts.xlsx')
hist_vol = pd.read_excel('all_data/realized_volatility.xlsx')

# Convert Date columns to datetime
ER['Date'] = pd.to_datetime(ER['Date'])
garch_vol['Date'] = pd.to_datetime(garch_vol['Date'])

In [3]:
# Optimization function
def mvo_optimize(expected_returns, covariance_matrix, bounds, init_guess, risk_aversion=5):
    def objective(weights):
        portfolio_return = np.dot(weights, expected_returns)
        portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))
        utility = portfolio_return - (risk_aversion/2) * portfolio_variance
        return -utility
    
    constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
    
    result = minimize(
        objective,
        init_guess,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints,
        options={'ftol': 1e-9}
    )
    
    if not result.success:
        print(f"Warning: Optimization failed - {result.message}")
        return None
    
    return result.x

# Bounds function
def volatility_based_bounds(volatilities, min_bound=0.001, max_bound=0.20, range_width=0.10):
    inv_vol = 1 / volatilities
    weights = inv_vol / inv_vol.sum()
    
    bounds = []
    for w in weights:
        lower = max(min_bound, w - range_width / 2)
        upper = min(max_bound, w + range_width / 2)
        bounds.append((lower, upper))
    return bounds

In [4]:
# Check ER structure
print("ER DATA:")
print(ER.head())
print(f"Shape: {ER.shape}")
print(f"Date range: {ER['Date'].min()} to {ER['Date'].max()}")
print(f"Columns: {ER.columns.tolist()}")

print("\n" + "="*50 + "\n")

# Check garch_vol structure
print("GARCH_VOL DATA:")
print(hist_vol.head())
print(f"Shape: {hist_vol.shape}")
print(f"Date range: {hist_vol['Date'].min()} to {hist_vol['Date'].max()}")
print(f"Columns: {hist_vol.columns.tolist()}")

ER DATA:
        Date      AAPL       ABT       AMD      AMZN      AVGO       AXP  \
0 2012-01-03  0.000127  0.001078 -0.001161 -0.004720 -0.003342 -0.001480   
1 2012-01-04  0.000980  0.001777  0.001357 -0.003747 -0.002725 -0.000097   
2 2012-01-05  0.001054  0.001630  0.000551 -0.004344 -0.002848 -0.000729   
3 2012-01-06  0.000979  0.001143 -0.000626 -0.004626 -0.004078 -0.001096   
4 2012-01-09  0.001395  0.001153 -0.000508 -0.003758 -0.004263 -0.001124   

        BAC     BRK-B       CAT  ...        PM      QCOM       RTX       TMO  \
0 -0.004291 -0.000344 -0.000678  ...  0.003079  0.001708 -0.001222 -0.002482   
1 -0.001765  0.000802  0.000924  ...  0.003491  0.002630  0.000141 -0.000674   
2 -0.002914 -0.000088  0.000544  ...  0.003064  0.001902 -0.000279 -0.001715   
3 -0.001538 -0.000457  0.000190  ...  0.002677  0.000207 -0.000981 -0.001752   
4 -0.000582 -0.000147  0.000314  ...  0.002528  0.000084 -0.000925 -0.000435   

       TMUS       UNH         V       WFC       WMT  

In [5]:
# Get date range
start_date = pd.to_datetime('2012-01-01')
end_date = ER['Date'].max()

# Storage for results
results_hist = []

current_start = start_date

while current_start <= end_date - relativedelta(months=2):
    # Define the 2-month estimation window end
    estimation_window_end = current_start + relativedelta(months=2)
    
    # Define target month start (month after estimation window)
    target_month_start = estimation_window_end
    
    # Get returns for EXACTLY 2-month estimation period
    returns_window = ER[(ER['Date'] >= current_start) & (ER['Date'] < estimation_window_end)].drop('Date', axis=1)
    
    # Skip if insufficient data
    if len(returns_window) < 20:
        current_start += relativedelta(months=1)
        continue
    
    # Calculate correlation matrix and expected returns from 2-month window
    R = returns_window.corr().values
    expected_returns = returns_window.mean().values
    
    # Get historical volatilities from 2-month window
    vol_hist = returns_window.std().values
    
    # Build covariance matrix
    D_hist = np.diag(vol_hist)
    Sigma_hist = D_hist @ R @ D_hist
    
    # Setup bounds and initial guess
    bounds = volatility_based_bounds(vol_hist)
    init_guess = (1 / vol_hist) / (1 / vol_hist).sum()
    
    # Optimize portfolio
    weights_hist = mvo_optimize(expected_returns, Sigma_hist, bounds, init_guess)
    
    # Store results if optimization succeeded
    if weights_hist is not None:
        stock_names = returns_window.columns.tolist()
        
        # Store historical weights
        result_hist = {'Date': target_month_start}
        for name, weight in zip(stock_names, weights_hist):
            result_hist[name] = weight
        results_hist.append(result_hist)
        
        # For display
        last_estimation_date = ER[ER['Date'] < estimation_window_end]['Date'].max()
        print(f"Estimation: {current_start.strftime('%Y-%m-%d')} to {last_estimation_date.strftime('%Y-%m-%d')} ({len(returns_window)} days) → Weights for {target_month_start.strftime('%Y-%m')}")
    
    # Move to next month
    current_start += relativedelta(months=1)

# Convert results to DataFrame
weights_hist_df = pd.DataFrame(results_hist)

Estimation: 2012-01-01 to 2012-02-29 (40 days) → Weights for 2012-03
Estimation: 2012-02-01 to 2012-03-30 (42 days) → Weights for 2012-04
Estimation: 2012-03-01 to 2012-04-30 (42 days) → Weights for 2012-05
Estimation: 2012-04-01 to 2012-05-31 (42 days) → Weights for 2012-06
Estimation: 2012-05-01 to 2012-06-29 (43 days) → Weights for 2012-07
Estimation: 2012-06-01 to 2012-07-31 (42 days) → Weights for 2012-08
Estimation: 2012-07-01 to 2012-08-31 (44 days) → Weights for 2012-09
Estimation: 2012-08-01 to 2012-09-28 (42 days) → Weights for 2012-10
Estimation: 2012-09-01 to 2012-10-31 (40 days) → Weights for 2012-11
Estimation: 2012-10-01 to 2012-11-30 (42 days) → Weights for 2012-12
Estimation: 2012-11-01 to 2012-12-31 (41 days) → Weights for 2013-01
Estimation: 2012-12-01 to 2013-01-31 (41 days) → Weights for 2013-02
Estimation: 2013-01-01 to 2013-02-28 (40 days) → Weights for 2013-03
Estimation: 2013-02-01 to 2013-03-28 (39 days) → Weights for 2013-04
Estimation: 2013-03-01 to 2013-04-

In [6]:
weights_hist_df.head()

,Date,AAPL,ABT,AMD,AMZN,AVGO,AXP,BAC,BRK-B,CAT,...,PM,QCOM,RTX,TMO,TMUS,UNH,V,WFC,WMT,XOM
0,2012-03-01,0.061585,0.011602,0.057022,0.001000,0.001000,0.001000,0.055492,0.001000,0.061573,...,0.001000,0.001000,0.001,0.037028,0.058460,0.001000,0.001000,0.075279,0.001000,0.001
1,2012-04-01,0.064934,0.001000,0.062246,0.001000,0.065915,0.001000,0.061963,0.013300,0.058687,...,0.001000,0.001000,0.001,0.064505,0.060071,0.001000,0.069006,0.001000,0.001000,0.001
2,2012-05-01,0.060354,0.001000,0.056814,0.001000,0.058504,0.081324,0.056513,0.001831,0.001000,...,0.070525,0.039153,0.001,0.001000,0.001000,0.001000,0.066027,0.075218,0.001000,0.001
3,2012-06-01,0.057726,0.084618,0.001000,0.061781,0.001000,0.066398,0.001000,0.001000,0.001000,...,0.067551,0.001000,0.001,0.001000,0.001000,0.071311,0.072108,0.063938,0.001000,0.001
4,2012-07-01,0.001000,0.080306,0.001000,0.063855,0.001000,0.063378,0.001000,0.088129,0.001000,...,0.001000,0.001000,0.001,0.001000,0.001000,0.083829,0.066851,0.001000,0.066097,0.001


In [7]:
print(f"\nTotal portfolios created: {len(weights_hist_df)}")
print(f"Date range: {weights_hist_df['Date'].min()} to {weights_hist_df['Date'].max()}")


Total portfolios created: 163
Date range: 2012-03-01 00:00:00 to 2025-09-01 00:00:00


In [8]:
weights_hist_df['Date'] = pd.to_datetime(weights_hist_df['Date']).dt.strftime('%m-%Y')
weights_hist_df.head()

,Date,AAPL,ABT,AMD,AMZN,AVGO,AXP,BAC,BRK-B,CAT,...,PM,QCOM,RTX,TMO,TMUS,UNH,V,WFC,WMT,XOM
0,03-2012,0.061585,0.011602,0.057022,0.001000,0.001000,0.001000,0.055492,0.001000,0.061573,...,0.001000,0.001000,0.001,0.037028,0.058460,0.001000,0.001000,0.075279,0.001000,0.001
1,04-2012,0.064934,0.001000,0.062246,0.001000,0.065915,0.001000,0.061963,0.013300,0.058687,...,0.001000,0.001000,0.001,0.064505,0.060071,0.001000,0.069006,0.001000,0.001000,0.001
2,05-2012,0.060354,0.001000,0.056814,0.001000,0.058504,0.081324,0.056513,0.001831,0.001000,...,0.070525,0.039153,0.001,0.001000,0.001000,0.001000,0.066027,0.075218,0.001000,0.001
3,06-2012,0.057726,0.084618,0.001000,0.061781,0.001000,0.066398,0.001000,0.001000,0.001000,...,0.067551,0.001000,0.001,0.001000,0.001000,0.071311,0.072108,0.063938,0.001000,0.001
4,07-2012,0.001000,0.080306,0.001000,0.063855,0.001000,0.063378,0.001000,0.088129,0.001000,...,0.001000,0.001000,0.001,0.001000,0.001000,0.083829,0.066851,0.001000,0.066097,0.001


#### **GARCH WEIGHTS**

In [9]:
# Check ER structure
print("ER DATA:")
print(ER.head())
print(f"Shape: {ER.shape}")
print(f"Date range: {ER['Date'].min()} to {ER['Date'].max()}")
print(f"Columns: {ER.columns.tolist()}")

print("\n" + "="*50 + "\n")

# Check garch_vol structure
print("GARCH_VOL DATA:")
print(garch_vol.head())
print(f"Shape: {garch_vol.shape}")
print(f"Date range: {garch_vol['Date'].min()} to {garch_vol['Date'].max()}")
print(f"Columns: {garch_vol.columns.tolist()}")

ER DATA:
        Date      AAPL       ABT       AMD      AMZN      AVGO       AXP  \
0 2012-01-03  0.000127  0.001078 -0.001161 -0.004720 -0.003342 -0.001480   
1 2012-01-04  0.000980  0.001777  0.001357 -0.003747 -0.002725 -0.000097   
2 2012-01-05  0.001054  0.001630  0.000551 -0.004344 -0.002848 -0.000729   
3 2012-01-06  0.000979  0.001143 -0.000626 -0.004626 -0.004078 -0.001096   
4 2012-01-09  0.001395  0.001153 -0.000508 -0.003758 -0.004263 -0.001124   

        BAC     BRK-B       CAT  ...        PM      QCOM       RTX       TMO  \
0 -0.004291 -0.000344 -0.000678  ...  0.003079  0.001708 -0.001222 -0.002482   
1 -0.001765  0.000802  0.000924  ...  0.003491  0.002630  0.000141 -0.000674   
2 -0.002914 -0.000088  0.000544  ...  0.003064  0.001902 -0.000279 -0.001715   
3 -0.001538 -0.000457  0.000190  ...  0.002677  0.000207 -0.000981 -0.001752   
4 -0.000582 -0.000147  0.000314  ...  0.002528  0.000084 -0.000925 -0.000435   

       TMUS       UNH         V       WFC       WMT  

In [10]:
# Get date range
start_date = pd.to_datetime('2012-03-01')
end_date = ER['Date'].max()

# Storage for results
results_garch = []

current_start = start_date

while current_start <= end_date - relativedelta(months=2):
    # Define target month (the month we're optimizing for)
    target_month_start = current_start
    target_month_end = current_start + relativedelta(months=1)
    
    # Get returns for 2-month estimation period BEFORE target month
    estimation_start = current_start - relativedelta(months=2)
    returns_window = ER[(ER['Date'] >= estimation_start) & (ER['Date'] < current_start)].drop('Date', axis=1)
    
    # Skip if insufficient data
    if len(returns_window) < 20:
        current_start += relativedelta(months=1)
        continue
    
    # Calculate correlation matrix and expected returns from 2-month window
    R = returns_window.corr().values
    expected_returns = returns_window.mean().values
    
    # Get GARCH volatilities for target month (e.g., March 2012 GARCH for March 2012)
    garch_target_month = garch_vol[(garch_vol['Date'] >= target_month_start) & (garch_vol['Date'] < target_month_end)]
    
    if garch_target_month.empty:
        current_start += relativedelta(months=1)
        continue
    
    # Average GARCH volatilities across all days in target month
    vol_garch = garch_target_month.drop('Date', axis=1).mean().values
    
    # Build covariance matrix using GARCH volatilities
    D_garch = np.diag(vol_garch)
    Sigma_garch = D_garch @ R @ D_garch
    
    # Setup bounds and initial guess (using GARCH volatilities)
    bounds = volatility_based_bounds(vol_garch)
    init_guess = (1 / vol_garch) / (1 / vol_garch).sum()
    
    # Optimize portfolio
    weights_garch = mvo_optimize(expected_returns, Sigma_garch, bounds, init_guess)
    
    # Store results if optimization succeeded
    if weights_garch is not None:
        stock_names = returns_window.columns.tolist()
        
        # Store GARCH weights
        result_garch = {'Date': target_month_start}
        for name, weight in zip(stock_names, weights_garch):
            result_garch[name] = weight
        results_garch.append(result_garch)
        
        # Display info
        garch_days = len(garch_target_month)
        estimation_end = current_start - relativedelta(days=1)
        print(f"Estimation: {estimation_start.strftime('%Y-%m-%d')} to {estimation_end.strftime('%Y-%m-%d')} | GARCH vol: {garch_target_month['Date'].min().strftime('%Y-%m-%d')} to {garch_target_month['Date'].max().strftime('%Y-%m-%d')} ({garch_days} days) → Weights for {target_month_start.strftime('%Y-%m')}")
    
    # Move to next month
    current_start += relativedelta(months=1)

# Convert results to DataFrame
weights_garch_df = pd.DataFrame(results_garch)

Estimation: 2012-01-01 to 2012-02-29 | GARCH vol: 2012-03-01 to 2012-03-30 (22 days) → Weights for 2012-03
Estimation: 2012-02-01 to 2012-03-31 | GARCH vol: 2012-04-02 to 2012-04-30 (20 days) → Weights for 2012-04
Estimation: 2012-03-01 to 2012-04-30 | GARCH vol: 2012-05-01 to 2012-05-31 (22 days) → Weights for 2012-05
Estimation: 2012-04-01 to 2012-05-31 | GARCH vol: 2012-06-01 to 2012-06-29 (21 days) → Weights for 2012-06
Estimation: 2012-05-01 to 2012-06-30 | GARCH vol: 2012-07-02 to 2012-07-31 (21 days) → Weights for 2012-07
Estimation: 2012-06-01 to 2012-07-31 | GARCH vol: 2012-08-01 to 2012-08-31 (23 days) → Weights for 2012-08
Estimation: 2012-07-01 to 2012-08-31 | GARCH vol: 2012-09-04 to 2012-09-28 (19 days) → Weights for 2012-09
Estimation: 2012-08-01 to 2012-09-30 | GARCH vol: 2012-10-01 to 2012-10-31 (21 days) → Weights for 2012-10
Estimation: 2012-09-01 to 2012-10-31 | GARCH vol: 2012-11-01 to 2012-11-30 (21 days) → Weights for 2012-11
Estimation: 2012-10-01 to 2012-11-30 

In [11]:
print(f"\nTotal portfolios created: {len(weights_garch_df)}")
print(f"Date range: {weights_garch_df['Date'].min()} to {weights_garch_df['Date'].max()}")


Total portfolios created: 161
Date range: 2012-03-01 00:00:00 to 2025-07-01 00:00:00


In [12]:
weights_garch_df['Date'] = pd.to_datetime(weights_garch_df['Date']).dt.strftime('%m-%Y')
weights_garch_df.head()

,Date,AAPL,ABT,AMD,AMZN,AVGO,AXP,BAC,BRK-B,CAT,...,PM,QCOM,RTX,TMO,TMUS,UNH,V,WFC,WMT,XOM
0,03-2012,0.069528,0.001000,0.061016,0.001000,0.001000,0.001000,0.060361,0.001000,0.066676,...,0.001000,0.001,0.001,0.001000,0.001000,0.069757,0.001000,0.065522,0.001000,0.001
1,04-2012,0.064821,0.001000,0.063043,0.001000,0.063011,0.001000,0.061383,0.001000,0.001000,...,0.001000,0.001,0.001,0.070216,0.063918,0.001000,0.067313,0.001000,0.001000,0.001
2,05-2012,0.065401,0.085246,0.062459,0.001000,0.001000,0.074398,0.063950,0.001000,0.001000,...,0.081494,0.001,0.001,0.001000,0.001000,0.066429,0.067196,0.069474,0.001000,0.001
3,06-2012,0.072384,0.089329,0.001000,0.064645,0.001000,0.069722,0.001000,0.001000,0.001000,...,0.082504,0.001,0.001,0.001000,0.001000,0.072088,0.001000,0.001000,0.073667,0.001
4,07-2012,0.001000,0.088330,0.001000,0.065246,0.001000,0.072182,0.001000,0.085052,0.001000,...,0.001000,0.001,0.001,0.001000,0.001000,0.069135,0.010179,0.001000,0.084903,0.001


In [13]:
weights_hist_df.to_excel("all_data/weights_hist.xlsx", index=False)
weights_garch_df.to_excel("all_data/weights_garch.xlsx", index=False)